In [ ]:
! mkdir -p datasets
%cd datasets
! wget -nc https://raw.githubusercontent.com/pablonoya/zigzag-ml/master/datasets/housing.csv
%cd ..

# Overfitting y Underfitting
Son dos problemas que pueden ocurrir luego de haber entrenado al modelo.  
El *overfitting*, o **sobreajuste**, ocurre cuando el modelo se ajusta demasiado a los datos de entrenamiento, como una curva que intenta cruzar por la mayor cantidad de puntos posible. Se lo conoce también como *high variance* o **varianza alta** .  
En cambio el *underfitting*, o ajuste débil, sucede cuando el modelo no se ajusta bien a los datos, como una recta que no puede alcanzar gran parte de los datos. Se lo conoce también como *high bias* o **sesgo elevado**.  

![Ajuste debil, ideal y sobreajuste](img/5.0.1_Overfitting_right_underfitting.png)

¿Recuerdas que una recta no bastaba? un polinomio de grado 3 fue mejor, y un polinomio de alto grado se ajusta demasiado a los datos, por el hecho de tener más curvas.

**Incluir más _features_ aumenta la complejidad** del modelo, más parámetros para entrenar dejan **mayor libertad** de valores para aprender, y si esta es alta podemos llegar al **sobreajuste**. Por el contrario, **reducir las features** causa que el modelo tenga **menos libertad** y aprenda de poca información, lo que deriva en **ajuste debil**.  

Debido a esto es **importante mezclar los datos y tener la mayor variedad posible** en nuestros conjuntos de entrenamiento y prueba 😉.

# Bias-variance trade-off
Esta situación que depende de las *features* nos indica que hay una **compensación** entre **varianza** y **_bias_**, incrementar la varianza disminuye el sesgo y viceversa 🔁.  
Un modelo con **alto bias** es más simple y realizará predicciones simples, como la **primera** gráfica.
Uno complejo tendrá **alta varianza** realizando predicciones más complejas que pueden ser muy variadas, como la **tercera** gráfica.

Por lo que debemos encontrar un **balance** entre ambos, el modelo no debe ser demasiado simple ni demasiado complejo, como en la **segunda** gráfica. Esto asegura que no perdamos el panorama general 😃.

# ¿Qué está mal en mi modelo?
Para saber a qué problema nos enfrentamos es necesario hacer pruebas con ambos conjuntos, tendremos **_overfitting_** cuando el **error en uno de los conjuntos sea mucho menor**, con frecuencia es el de entrenamiento 😨.  
Y **_underfitting_** cuando el **error de ambos conjuntos sea elevado** 😰.

## Solucionando el underfitting
La recomendación general es **incrementar la complejidad del modelo**, esto se logra añadiendo más features, o subiendo el grado polinómico al que elevemos algunas.  
Es importante tomar en cuenta que **las features que añadamos deben estar siempre disponibles**, caso contrario, no podremos utilizar el modelo en la vida real ☹. 

## Solucionando el overfit
Lo contrario sería **disminuir la complejidad**, pero aquí tenemos más opciones, como **entrenar utilizando mayor cantidad de datos** lo cual ayuda a no perder el objetivo general, o usar **regularización**, el nombre se debe a que regularemos la libertad de los coeficientes para que **no sean tan altos**, reduciendo la varianza.

![regularización](img/5.3.2_Regularization.png)

Existen ~~dos~~ tres tipos de regularización: $L_2$ o Ridge, $L_1$ o Lasso, y Elastic net, la combinación de ambas. Todas se aplican sobre la regresión lineal 😉.

## Ridge Regularization
Esta regularización minimiza los coeficientes acercándolos a cero, pero nunca llegan a cero.
Tenemos esta regularización en el objeto `Ridge`, es parecido a `LinearRegression` pero recibe el parámetro `alpha` que va de 0 a 1, mientras más alto sea, mayor será la restricción sobre el modelo.

Seguiremos el mismo procedimiento que antes: cargar los datos, seleccionar features, dividir en train y test para finalmente entrenar dos modelos y comparar sus coeficientes, pero esta vez entrenaremos sobre una pequeña muestra de 20 datos, porque utilizar más soluciona el overfitting 😄.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(42)
data_housing = pd.read_csv('./datasets/housing.csv')
data_housing.dropna(inplace=True)

X = data_housing['median_income']
y = data_housing['median_house_value']

# podemos tomar un número específico si pasamos un entero
X_train, _, y_train, _ = train_test_split(X.values.reshape(-1, 1), y, train_size=20)

In [ ]:
# importamos también Ridge
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures

# exageremos el grado para incrementar complejidad
poly = PolynomialFeatures(degree=4, include_bias=False)
X_train_poly = poly.fit_transform(X_train)

# tendremos 2 modelos para comparar
model = LinearRegression()
model_l2 = Ridge(alpha=1)

model.fit(X_train_poly, y_train)
model_l2.fit(X_train_poly, y_train)

print(f"Coeficientes Linear {model.coef_}")
print(f"Coeficientes Ridge  {model_l2.coef_}")

Como ves, los coeficientes han sido reducidos, pero ninguno ha desaparecido.
Esto modifica al modelo de la siguiente manera:

In [ ]:
import matplotlib.pyplot as plt

xpoints = np.linspace(min(X_train[:, 0]), max(X_train[:, 0]), 30)
xpoints_poly = poly.transform(xpoints.reshape(-1, 1))
ypoints = model.predict(xpoints_poly)

plt.figure(figsize=(8,5))
plt.scatter(X_train[:, 0], y_train, color="grey")

# el parármetro color define el color de la linea
plt.plot(xpoints, ypoints, color="red", label="Linear")
plt.plot(xpoints, model_l2.predict(xpoints_poly), color="blue", linewidth="2", label="Ridge")

plt.legend(loc="best")

La regularización ha restringido la curva, ya no se ajusta demasiado a los datos.
El parámetro `linewitdh` cambia el grosor, y `label` pone etiquetas a las gráficas, pero para mostrarlas debemos llamar a `plt.legend` el parámetro `loc` con el valor "best" tratará de determinar la mejor ubicación para las etiquetas.

## Lasso Regularization
El objeto `Lasso` es idéntico a `Ridge` con la diferencia de que **sí puede** reducir los coeficientes menos importantes a cero. Esto nos puede ayudar a decidir qué features son menos importantes

In [ ]:
from sklearn.linear_model import Lasso

model_l1 = Lasso(alpha=1)
model_l1.fit(X_train_poly, y_train)

print(f"Coeficientes Linear {model.coef_}")
print(f"Coeficientes Lasso  {model_l1.coef_}")

plt.scatter(X_train[:, 0], y_train, color="grey")

# tambien puedes usar la letra inicial de ciertos colores
plt.plot(xpoints, ypoints, color="r", label="Linear")
plt.plot(xpoints, model_l1.predict(xpoints_poly), color="b", linewidth="2", label="Lasso")

plt.legend(loc="best")

La curva se ha aplanado más, y en este caso una curva es de hecho más adecuada que una recta, pero quizá bastaría una de grado 3.

## Elastic Net
Pretende equilibrar a Lasso combinándolo con Ridge, e incluye el parámetro `l1_ratio` que determina cuánta importancia, entre 0 y 1, se le dará a Lasso. Por tanto Ridge tendrá el **complemento** de ese parámetro, si `l1_ratio` es 0.3 entonces L2 tendrá 0.7

In [ ]:
from sklearn.linear_model import ElasticNet

model_elastic_net = ElasticNet(alpha=0.5, l1_ratio=0.4)
model_elastic_net.fit(X_train_poly, y_train)

print(f"Coeficientes Ridge (L2)   {model_l2.coef_}")
print(f"Coeficientes Lasso (L1)   {model_l1.coef_}")
print(f"Coeficientes Elastic net  {model_elastic_net.coef_}")

Elastic Net sólo ha reducido un coeficiente a cero, de esta manera controlamos que no se eliminen varias features, por esto puede ser preferible usarla en lugar de Lasso, pero tenemos otro parámetro que determinar.

# Ejercicios
Grafica la curva de Elastic Net, luego varía el valor alfa de todos los modelos, ¿Qué ocurre con valores bajos?

In [ ]:
# misma gráfica, diferente modelo


Provoca deliberadamente (como en este capítulo) *overfitting* o *underfitting* con varias features, luego aplica las correcciones necesarias y evalúa el resultado con métricas. 

In [ ]:
# revisa los dos capítulos anteriores


# Parámetros e Hiperparámetros
Hasta ahora he llamado "parámetro" a dos términos que son muy parecidos: los **coeficientes del modelo** y los **argumentos** de las funciones y objetos que usamos. Pero tienen una diferencia fundamental:

La lista de coeficientes es **determinada por el modelo**, esto en *machine learning* se sigue conociendo como **parámetro** 😄.  

Aquello que **no determina el modelo** es un **hiperparámetro**, como las features elegidas, el porcentaje de división entre train y test, el grado de las features polinomiales, el coeficiente de regularización o el ratio entre $L_1$ y $L_2$ para Elastic Net.

Aprenderemos más hiperparámetros según aprendamos más conceptos, y uno de los **hiper**parámetros más importantes es el *learning rate* o **ratio de aprendizaje**, este controla qué tanto se ajusta un modelo por cada iteración del algoritmo que permite a un modelo aprender 🧠.

Para la regresión lineal, el algoritmo es *Gradient Descent* o [Descenso del gradiente](6_Descenso_del_gradiente.ipynb).